In [ ]:
# Libaries
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
import leafmap.foliumap as leafmap

In [ ]:
# Directory
current = Path.cwd()
parent = current.resolve().parent / 'data'
os.chdir(parent)
os.getcwd()

In [ ]:
# Data
ZipCodes = gpd.read_file("cb_2019_us_zcta510_500k/cb_2019_us_zcta510_500k.shp")
MODZCTA10 = gpd.read_file("Modified Zip Code Tabulation Areas (MODZCTA)_20240710/geo_export_39ff3478-bd33-4802-a75e-cd3a5fc72c34.shp")
OCS_Oil19 = pd.read_csv("Office_of_Climate_and_Sustainability__2019_Fuel_Oil_Sales_20240710.csv")
ZCTA_XWalk = pd.read_csv("ZCTACrosswalkMN.csv")

In [ ]:
# Data processing

ZipCodes['ZCTA5CE10'] = ZipCodes['ZCTA5CE10'].astype(int)
MNZip_Geo = ZipCodes[(ZipCodes['ZCTA5CE10'].between(10001, 10282))]
MNZip_Geo = MNZip_Geo.rename(columns={'ZCTA5CE10':'ZCTA'})
Oil19 = OCS_Oil19[(OCS_Oil19['Zip Code'].between(10001, 10282))]
Oil19 = Oil19.rename(columns={'Zip Code': 'ZCTA'})
Oil19_Geo = pd.merge(MNZip_Geo, Oil19, on='ZCTA', how='left')
MainOil19_Geo = Oil19_Geo[['ZCTA',
       'Total #2 Fuel Oil   without Biodiesel Component',
       'Total Biodiesel Blended with #2 Fuel Oil',
       'Total #4 Fuel Oil without Biodiesel Component',
       'Total Biodiesel Blended with #4 Fuel Oil'
                   ]]
#MainOil19_Geo = MainOil19_Geo[['Total #2 Fuel Oil without Biodiesel Component',
#       'Total Biodiesel Blended with #2 Fuel Oil',
#       'Total #4 Fuel Oil without Biodiesel Component',
#       'Total Biodiesel Blended with #4 Fuel Oil']].round(2)

MainOil19_Mod = pd.merge(ZCTA_XWalk, MainOil19_Geo, on ='ZCTA', how ='left')

MainOil19_Group = MainOil19_Mod.groupby('MODZCTA').agg({'Total #2 Fuel Oil without Biodiesel Component': 'sum',
                                                 'Total Biodiesel Blended with #2 Fuel Oil': 'sum',
                                                 'Total #4 Fuel Oil without Biodiesel Component': 'sum',
                                                 'Total Biodiesel Blended with #4 Fuel Oil': 'sum'}
).reset_index()
MODZCTA10 = MODZCTA10.rename(columns={'modzcta': 'MODZCTA'})
MODZCTA10['MODZCTA'] = MODZCTA10['MODZCTA'].astype(int)
MainOil19_GroupGeo = pd.merge(MainOil19_Group, MODZCTA10, on='MODZCTA', how ='left')
MainOil19_GroupGeo = gpd.GeoDataFrame(data=MainOil19_GroupGeo, geometry='geometry')

Fuel2_19 = Oil19_Geo[['ZCTA',
       '#2 Fuel Oil with 0% biodiesel (gallons)',
       '#2 Fuel Oil with 2% biodiesel (gallons)',
       '#2 Fuel Oil with 5% biodiesel (gallons)',
       '#2 Fuel Oil with 6% biodiesel (gallons)',
       '#2 Fuel Oil with 10% biodiesel (gallons)',
       '#2 Fuel Oil with 17% biodiesel (gallons)',
       'Total #2 Fuel Oil without Biodiesel Component',
       'Total Biodiesel Blended with #2 Fuel Oil'
                   ]]
Fuel4_19 = Oil19_Geo[['ZCTA',
       '#4 Fuel Oil with 2% biodiesel (gallons)',
       '#4 Fuel Oil with 5% biodiesel (gallons)',
       '#4 Fuel Oil with 6% biodiesel (gallons)',
       'Total #4 Fuel Oil without Biodiesel Component',
       'Total Biodiesel Blended with #4 Fuel Oil'
                   ]]

In [ ]:
# Returning GeoJSONs
MainOil19_GroupGeo.to_file('FuelOil_MN_19.geojson', driver='GeoJSON')

In [6]:
# Plotting
m = leafmap.Map(center=[40.7248387,-73.9775537], zoom=12)
m.add_data(MainOil19_GroupGeo, column='Total Biodiesel Blended with #2 Fuel Oil', scheme='Quantiles', 
           cmap='Blues', legend_title="Total Biodiesel with #2 Fuel (gallons)"
)
m.add_data(MainOil19_GroupGeo, column='Total Biodiesel Blended with #4 Fuel Oil', scheme='Quantiles', 
           cmap='Blues', legend_title="Total Biodiesel with #4 Fuel (ga#llons)"
)
#m.show_in_browser()